In [ ]:
import os
import configparser

# Loading configurations
configParser = configparser.RawConfigParser()   
configFilePath = r'configuration.txt'
configParser.read(configFilePath)

insert_amd_env_vars =  int(configParser.get('COMMON', 'insert_amd_env_vars'))
HSA_OVERRIDE_GFX_VERSION =  configParser.get('COMMON', 'HSA_OVERRIDE_GFX_VERSION')
ROCM_PATH =  configParser.get('COMMON', 'ROCM_PATH')

if(insert_amd_env_vars != 0):
    os.environ["HSA_OVERRIDE_GFX_VERSION"] = HSA_OVERRIDE_GFX_VERSION
    os.environ["ROCM_PATH"] = ROCM_PATH
    
#import os
#os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
#os.environ["AUTOGRAPH_VERBOSITY"] = "0"

In [ ]:

import configparser

# Loading configurations
configParser = configparser.RawConfigParser()   
configFilePath = r'configuration.txt'
configParser.read(configFilePath)


datasetPathVideo =  configParser.get('COMMON', 'test_datasetPathVideo')

datasetPathFeatures =  configParser.get('evaluate_imagen', 'test_datasetPathFeatures')
datasetPathGeneratedFaces =  configParser.get('evaluate_imagen', 'test_datasetPathGeneratedFaces')

datasetPathDatabase =  configParser.get('COMMON', 'test_datasetPathDatabase') + '/dataset.db'

ttwbdf =  int(configParser.get('evaluate_imagen', 'time_to_wait_before_deleting_files'))


cuda =  int(configParser.get('COMMON', 'cuda'))
cpus =  int(configParser.get('COMMON', 'cpus'))




In [ ]:
#TODO document jupyter
import pickle
import sqlite3 as sl
import pandas as pd
import numpy as np
from PIL import Image

from textwrap import wrap
import os


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd



In [ ]:
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None


In [ ]:
image_size =  int(configParser.get('COMMON', 'resizeImageTo'))
begin_with_image_size = int(configParser.get('COMMON', 'begin_with_image_size'))
timesteps= int(configParser.get('COMMON', 'timesteps') )
unet1_image_size =  int(configParser.get('COMMON', 'unet1_image_size'))
audio_length_used =  configParser.get('evaluate_imagen', 'audio_length_used') 
model_filename =  configParser.get('evaluate_imagen', 'model_filename') + '_'  + audio_length_used +  's.pt'
openl3_mode =  configParser.get('evaluate_imagen', 'openl3_mode')
folder =  configParser.get('evaluate_imagen', 'folder')
audio_embs =  configParser.get('COMMON', 'audio_embs') 
unet_dim =  int(configParser.get('COMMON', 'unet_dim'))

In [ ]:
ignore_speaker_embedding = bool(int(configParser.get('train_imagen', 'ignore_speaker_embedding') ))
ignore_speech_brain = bool(int(configParser.get('train_imagen', 'ignore_speech_brain') ))
ignore_pyannote_titanet_speakernet = bool(int(configParser.get('train_imagen', 'ignore_pyannote_titanet_speakernet') ))
ignore_audio_features = bool(int(configParser.get('train_imagen', 'ignore_audio_features') ))
ignore_pyAudioAnalysis = bool(int(configParser.get('train_imagen', 'ignore_pyAudioAnalysis') ))
ignore_librosa = bool(int(configParser.get('train_imagen', 'ignore_librosa') ))
ignore_image_guide = bool(int(configParser.get('train_imagen', 'ignore_image_guide') ))
ignore_additional_attributes = bool(int(configParser.get('train_imagen', 'ignore_additional_attributes') ))
ignore_age = bool(int(configParser.get('train_imagen', 'ignore_age') ))
ignore_gender = bool(int(configParser.get('train_imagen', 'ignore_gender') ))
ignore_ethnicity = bool(int(configParser.get('train_imagen', 'ignore_ethnicity') ))
ignore_language_spoken = bool(int(configParser.get('train_imagen', 'ignore_language_spoken') ))

In [ ]:
skip_speaker_embedding = bool(int(configParser.get('evaluate_imagen', 'skip_speaker_embedding') ))
skip_speech_brain = bool(int(configParser.get('evaluate_imagen', 'skip_speech_brain') ))
skip_pyannote_titanet_speakernet = bool(int(configParser.get('evaluate_imagen', 'skip_pyannote_titanet_speakernet') ))
skip_audio_features = bool(int(configParser.get('evaluate_imagen', 'skip_audio_features') ))
skip_pyAudioAnalysis = bool(int(configParser.get('evaluate_imagen', 'skip_pyAudioAnalysis') ))
skip_librosa = bool(int(configParser.get('evaluate_imagen', 'skip_librosa') ))
skip_image_guide = bool(int(configParser.get('evaluate_imagen', 'skip_image_guide') ))
skip_additional_attributes = bool(int(configParser.get('evaluate_imagen', 'skip_additional_attributes') ))
skip_age = bool(int(configParser.get('evaluate_imagen', 'skip_age') ))
skip_gender = bool(int(configParser.get('evaluate_imagen', 'skip_gender') ))
skip_ethnicity = bool(int(configParser.get('evaluate_imagen', 'skip_ethnicity') ))
skip_language_spoken = bool(int(configParser.get('evaluate_imagen', 'skip_language_spoken') ))

In [ ]:
def speaker_emb_preprocess(speaker_emb2):

    if(ignore_speaker_embedding or ignore_speech_brain 
       or skip_speaker_embedding or skip_speech_brain):
        #print('ignore speech brain')
        return np.zeros((1,768)).tolist()
    else:
        speaker_emb2 = pickle.loads(speaker_emb2)
        #print(speaker_emb2.print)
        speaker_emb2 = speaker_emb2.squeeze()
        speaker_emb2 = np.pad(speaker_emb2, (288), 'constant', constant_values=(0))
        speaker_emb2 = np.tile(speaker_emb2, (1, 1))
        speaker_emb2 = speaker_emb2 / 200.0
        #print(speaker_emb2.shape)
        speaker_emb2 = np.array(speaker_emb2).tolist()
        return speaker_emb2


def audio_emb_preprocess2(speaker_emb2):
    
    if(ignore_pyannote_titanet_speakernet or ignore_speaker_embedding
       or skip_pyannote_titanet_speakernet or skip_speaker_embedding):
        #print('ignore speech brain')
        return np.zeros((28,768)).tolist()
    else:
        speaker_emb2 = pickle.loads(speaker_emb2)
        a = speaker_emb2 #np.zeros(shape=(24, 512))
        a = a[-28:,:] # Actually only last 39 are the relevant ones
        #print('au')
        #print(a.shape)
        b = np.zeros(shape=(a.shape[0], 768-a.shape[1]))
        #c = np.zeros(shape=(28-a.shape[0], 768))
        arr = np.concatenate((a, b), axis=1)
        #arr = np.concatenate((arr, c), axis=0)
        arr = arr / (1 if(audio_embs == 'wav2vec') else 10 if(audio_embs == 'openl3')  else 1 if(audio_embs == 'pyannoteTitaNet') else 1)
        #print(str(arr.max()))
        #print(arr.shape)
        speaker_emb2 = np.array(arr).tolist()
        return speaker_emb2



def audio_features_preprocess(audio_features):
    #  79 belong to pyaudioanalysis
    # 111 belogn to liborsa
    if(ignore_audio_features or skip_audio_features):
        #print('ignore speech brain')
        return np.zeros((190,768)).tolist()
    else:
        #print(video_id)
        audio_features = pickle.loads(audio_features)
        #print(audio_features.shape) # 190 x 128
        audio_features = audio_features[0:190]
        
        #import sys
        #np.set_printoptions(threshold=sys.maxsize)
        #print(np.argwhere(audio_features == 0))

        if((ignore_pyAudioAnalysis and not ignore_librosa)
           or (skip_pyAudioAnalysis and not skip_librosa)):
            zpa = np.zeros((190-110,128))
            audio_features = audio_features[80:190]
            audio_features = np.vstack((audio_features,zpa))
            #print('ignore pyAudioAnalysis')
            #print(audio_features.shape)
        elif((ignore_librosa and not ignore_pyAudioAnalysis)
             or(skip_librosa and not skip_pyAudioAnalysis)):
            zpa = np.zeros((190-80,128))
            audio_features = audio_features[0:80]
            audio_features = np.vstack((audio_features,zpa))
            #print('ignore librosa')
            #print(audio_features.shape)
        else:
            audio_features = np.zeros((190,128))
            #print('ignore pyaudioanalysis and librosa')


        z1 = np.zeros((190,768-128))
        audio_features = np.hstack((audio_features,z1))
        audio_features = audio_features / 100.0
        #print(audio_features.shape)
    return audio_features



import random

def make_square(im, min_size=image_size, fill_color=(0, 0, 0, 0)):
    x, y = im.size
    size = max(min_size, x, y)
    new_im = Image.new('RGB', (size, size), fill_color)
    new_im.paste(im, (int((size - x) / 2), int((size - y) / 2)))
    im.close()
    return new_im

def image_guide_preprocess_low_res_dummy(path):
    #print(face_id)

    image_guide = np.zeros((1, 768))
    return image_guide

def image_guide_preprocess_low_res(path):
    #print(face_id)


    if(random.random() > 2): #never
        image_guide = np.zeros((49, 768))
        #print(image_guide.shape)
    else:

        image = Image.open(path).convert('RGB')

        w_s = image_size / (1+2 * 0.5)
        h_s = image_size / (1+2 * 0.4)

        image = image.crop((0.4*w_s, 0.0*h_s, 1.6*w_s, 1.4*h_s))

        image = make_square(image)

        image = image.resize((begin_with_image_size,begin_with_image_size))
        im = image
        

        #print('saving')
        #image.save('opop.png')

        #print(np.array(image,np.float32).shape)

        pix = np.array(image, np.float32)
        pix = np.moveaxis(pix, -1, 0)

        pix = pix / 255
        image.close()
        im.close()
        return pix.tolist()
    return image_guide


def image_guide_preprocess(image_guide):

    image_guide = pickle.loads(image_guide)
    print(image_guide.shape)
    image_guide = image_guide.squeeze() #197 x 768
    image_guide = image_guide[1::4] #49 x 768
    return image_guide

from PIL import Image,ImageFilter
import random
#import cv2
import numpy as np
#from matplotlib import pyplot as plt



In [ ]:
import random


def process_age(age):
    if(ignore_age or skip_age):
        x = np.zeros(768)
        x[767] = 1
        return x
    elif(age > 100 or age < 0):
        x = np.zeros(768)
        x[767] = 1
        return x
    else:
        x = np.ones(768) * (age / 100.0)
        x[767] = 0
        return x

def process_gender(gender):
    if(ignore_gender or skip_gender):
        return np.zeros(768) 
    elif(gender == 'man'):
        return np.ones(768)
    elif(gender == "woman"):
        return np.ones(768) * -1
    else:
        return np.zeros(768)
    
# TODO
def process_ethnicity(eth):
    x = np.zeros(768)
    if(ignore_ethnicity or skip_ethnicity):
        x = x
    elif(eth == "indian"):
        x[0] = 1
    elif(eth == "asian"):
        x[16]=1
    elif(eth == "latino hispanic"):
        x[2]=1
    elif(eth == "black"):
        x[3]=1
    elif(eth == "middle eastern"):
        x[4]=1
    elif(eth == "white"):
        x[5]=1 
    else:
        x = x
    return x     

def process_language(lan):
    x = np.zeros(768)
    if(ignore_language_spoken or skip_language_spoken):
        x = x
    elif(lan == "Arabic"):
        x[0] = 1
    elif(lan == "Portuguese"):
        x[16]=1
    elif(lan == "Romansh_Sursilvan"):
        x[2]=1
    elif(lan == "Japanese"):
        x[3]=1
    elif(lan == "Ukranian"):
        x[4]=1
    elif(lan == "German"):
        x[5]=1   
    elif(lan == "Chinese_China"):
        x[6]=1   
    elif(lan == "Welsh"):
        x[7]=1  
    elif(lan == "English"):
        x[8]=1
    elif(lan == "Kabyle"):
        x[9]=1 
    elif(lan == "Kyrgyz"):
        x[10]=1
    elif(lan == "Georgian"):
        x[11]=1
    elif(lan == "Persian"):
        x[12]=1 
    elif(lan == "French"):
        x[13]=1
    elif(lan == "Interlingua"):
        x[14]=1
    elif(lan == "Swedish"):
        x[15]=1
    elif(lan == "Spanish"):
        x[16]=1 
    elif(lan == "Dhivehi"):
        x[17]=1
    elif(lan == "Kinyarwanda"):
        x[18]=1 
    elif(lan == "Tatar"):
        x[19]=1
    elif(lan == "Hakha_Chin"):
        x[20]=1 
    elif(lan == "Tamil"):
        x[21]=1 
    elif(lan == "Greek"):
        x[22]=1
    elif(lan == "Latvian"):
        x[23]=1 
    elif(lan == "Russian"):
        x[24]=1
    elif(lan == "Breton"):
        x[25]=1
    elif(lan == "Catalan"):
        x[26]=1    
    elif(lan == "Maltese"):
        x[27]=1 
    elif(lan == "Slovenian"):
        x[28]=1    
    elif(lan == "Indonesian"):
        x[29]=1    
    elif(lan == "Dutch"):
        x[30]=1
    elif(lan == "Chinese_Taiwan"):
        x[31]=1 
    elif(lan == "Sakha"):
        x[32]=1 
    elif(lan == "Polish"):
        x[33]=1 
    elif(lan == "Czech"):
        x[34]=1 
    elif(lan == "Romanian"):
        x[35]=1 
    elif(lan == "Mangolian"):
        x[36]=1 
    elif(lan == "Italian"):
        x[37]=1 
    elif(lan == "Chinese_Hongkong"):
        x[38]=1 
    elif(lan == "Estonian"):
        x[39]=1 
    elif(lan == "Basque"):
        x[40]=1 
    elif(lan == "Esperanto"):
        x[41]=1 
    elif(lan == "Frisian"):
        x[42]=1 
    elif(lan == "Turkish"):
        x[43]=1 
    elif(lan == "Chuvash"):
        x[44]=1 
    else:
        x = x
    return x        

In [ ]:
def process_input(age,ethnicity,gender,language,speaker_emb,audio_emb,audio_features,image_guide):
    speaker_emb = np.asarray(speaker_emb, dtype=np.float32)
    speaker_emb = speaker_emb.squeeze()
    audio_emb = np.asarray(audio_emb, dtype=np.float32)
    audio_emb = audio_emb.squeeze()
    #print(speaker_emb.shape)
    h = np.vstack((age, ethnicity))
    h = np.vstack((h, gender))
    h = np.vstack((h, language))
    h = np.vstack((h, speaker_emb))
    h = np.vstack((h, audio_emb))
    h = np.vstack((h,audio_features))
    h = np.vstack((h,image_guide))
    #print('aaaaaaaaaaa')
    #print(h.shape)
    j = np.zeros(768)
    j = np.tile(j,(256-h.shape[0],1))
    h = np.vstack((h, j))
    #print(h.shape)
    return h.tolist()


In [ ]:
import lancedb
uri = datasetPathDatabase
db = lancedb.connect(uri)

table = db.open_table("video_stage1")

In [ ]:
import torch
from imagen_pytorch import Unet, Imagen, ImagenTrainer,NullUnet
from imagen_pytorch.data import Dataset

unet0 = NullUnet()  # add a placeholder "null" unet for the base unet

unet1 = Unet(
    dim = unet_dim,
    cond_dim = 512,
    dim_mults = (1, 2, 4, 8),
    num_resnet_blocks = 3,
    layer_attns = (False, True, True, True),
    layer_cross_attns = (False, True, True, True)
)

unet2 = Unet(
    dim = unet_dim,
    cond_dim = 512,
    dim_mults = (1, 2, 4, 8),
    num_resnet_blocks = (2, 4, 8, 8),
    layer_attns = (False, False, False, True),
    layer_cross_attns = (False, False, False, True)
)

#unet = Unet(
#    dim = 32,
#    dim_mults = (1, 2, 4, 8),
#    num_resnet_blocks = 1,
#    layer_attns = (False, False, False, True),
#    layer_cross_attns = False
#)

# imagen, which contains the unet above

#imagen = Imagen(
#    unets = unet,
#    image_sizes = 32,
#    timesteps = 1000
#)

imagen = Imagen(
    unets = (unet0,unet1, unet2),
    image_sizes = (begin_with_image_size,unet1_image_size, image_size),
    timesteps = timesteps,
    cond_drop_prob = 0.1
).cuda()

trainer = ImagenTrainer(
    imagen = imagen,
    split_valid_from_train = True # whether to split the validation dataset from the training
).cuda()

print('Using model file ' + model_filename)
trainer.load(model_filename)





In [ ]:
import pickle
import torch
from torch.utils.data import TensorDataset, DataLoader
import pathlib




for index, row in table.to_pandas().iterrows():

    absPathVideo = row['video_path']
    absPathFace = row['face_path']

    data_frame = None
    

    with open(row['features_path'], 'rb') as pickle_file:
        data_frame = pickle.load(pickle_file)

    data_frame['low_res_image'] = absPathFace

    data_frame['SPEAKER_EMB'] = data_frame['SPEAKER_EMB'].apply(lambda x: speaker_emb_preprocess(x))
    data_frame['AUDIO_EMB'] = data_frame['AUDIO_EMB'].apply(lambda x: audio_emb_preprocess2(x))
    data_frame['AUDIO_FEATURES'] = data_frame['AUDIO_FEATURES'].apply(lambda x: audio_features_preprocess(x))
    data_frame['image_guide'] = data_frame['image_guide'].apply(lambda x: image_guide_preprocess_low_res_dummy(x))
    data_frame['low_res_image'] = data_frame['low_res_image'].apply(lambda x: image_guide_preprocess_low_res(x))
    data_frame['caption_a'] = data_frame['caption_a'].apply(lambda x: process_age(x))
    data_frame['caption_g'] = data_frame['caption_g'].apply(lambda x: process_gender(x))
    data_frame['caption_l'] = data_frame['caption_l'].apply(lambda x: process_language(x))
    data_frame['caption_e'] = data_frame['caption_e'].apply(lambda x: process_ethnicity(x))

    data_frame['INPUT'] = data_frame['SPEAKER_EMB']

    for index, row in data_frame.iterrows():
        x = process_input(data_frame.loc[index,"caption_a"],data_frame.loc[index,"caption_e"]
                            ,data_frame.loc[index,"caption_g"],
                        data_frame.loc[index,"caption_l"],data_frame.loc[index,"SPEAKER_EMB"],
                        data_frame.loc[index,"AUDIO_EMB"],data_frame.loc[index,"AUDIO_FEATURES"],
                        data_frame.loc[index,"image_guide"])
        x = [x]
        #AADFS = AADFS
        data_frame.loc[index,"INPUT"] = x

    data_frame = data_frame.drop(['caption_e', 'caption_g','caption_l','SPEAKER_EMB','AUDIO_EMB'], axis=1)

    
    

    input = data_frame['INPUT'].to_numpy()
    input = np.array([np.array(xi) for xi in input])
    input[np.isnan(input)] = 0
    input[input > 10] = 10
    input[input < -10] = -10

    input2 = data_frame['low_res_image'].to_numpy()
    input2 = np.array([np.array(xi) for xi in input2])

    print("Data Preprocessed...")

    input = input.squeeze().squeeze()
    input2 = input2.squeeze().squeeze()

    input = torch.from_numpy(input)
    input = input.to(torch.float)

    input2 = torch.from_numpy(input2)
    input2 = input2.to(torch.float)

    input = input.squeeze()
    input = input[None,:,:]

    input2 = input2.squeeze()
    input2 = input2[None,:,:]


    absPathGeneratedFace = absPathVideo.replace(datasetPathVideo,datasetPathGeneratedFaces)
    absPathGeneratedFace = os.path.splitext(absPathGeneratedFace)[0]
    absPathGeneratedFace = absPathGeneratedFace + "_generated_face_"+ ".png"
    #print(absPathFace)
    #Create Directory
    pathlib.Path(os.path.dirname(absPathGeneratedFace)).mkdir(parents=True, exist_ok=True) 

    images = trainer.sample(text_embeds=input,start_image_or_video = input2,
                        stop_at_unet_number=3,start_at_unet_number = 2
                        ,batch_size = 1, return_pil_images = True) # returns List[Image]
    
    images[0].save(absPathGeneratedFace)

    



In [ ]:
#import os
#os._exit(0)